# Temporal model Deep GRU + Shallow GRU


In [6]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, SimpleRNN

In [7]:
import gc
gc.collect()
tf.keras.backend.clear_session()
from numba import cuda 
device = cuda.get_current_device()
device.reset()

## Helper functions

In [8]:
def load_embeddings(subject):
    path = '../embeddings/embeddings_' + subject
    normal_embs = np.load(path + '_normal.npy')
    sleepy_embs = np.load(path + '_sleepy.npy')

    return normal_embs, sleepy_embs

def load_multi_embeddings(subjects):
    normal_dict = {}
    sleepy_dict = {}

    for sub in subjects:
        path = '../embeddings/embeddings_sub' + str(sub)
        normal_frames = np.load(path + '_normal.npy')
        sleepy_frames = np.load(path + '_sleepy.npy')

        normal_dict[str(sub)] = normal_frames
        sleepy_dict[str(sub)] = sleepy_frames

    return normal_dict, sleepy_dict

## First model: GRU deep features

### Model initiation

In [4]:
import gc
gc.collect()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.callbacks import EarlyStopping

class DrowsinessDetector:
    def __init__(self, segment_length):
        self.segment_length = segment_length
        self.input_shape = (self.segment_length, 2048)
        
    def split_frames(self, frames):
        num_segments = len(frames) // self.segment_length
        frames = frames[:num_segments * self.segment_length]
        return np.array(np.split(frames, num_segments))
    
    def create_labels(self, num_segments, label):
        return np.array([label] * num_segments)
    
    def shuffle_data(self, X, y):
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        return X[indices], y[indices]

    def construct_model(self):
        model = Sequential()
        model.add(SimpleRNN(32, input_shape=self.input_shape))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def get_deep_data(self, normal_frames_dict, sleepy_frames_dict, subjects, leave_out_subjects):
            X_train, y_train = None, None
            for subject in subjects:
                if subject in leave_out_subjects:
                    continue
                print('train', subject)
                frames = normal_frames_dict[subject]
                X = self.split_frames(frames)
                print(X.shape)
                assert 0
                y = self.create_labels(len(X), 0)
                if X_train is None:
                    X_train, y_train = X, y
                else:
                    X_train = np.concatenate((X_train, X))
                    y_train = np.concatenate((y_train, y))

                frames = sleepy_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 1)
                X_train = np.concatenate((X_train, X))
                y_train = np.concatenate((y_train, y))

            X_test, y_test = None, None
            for subject in leave_out_subjects:
                print('test', subject)
                frames = normal_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 0)

                if X_test is None:
                    X_test, y_test = X, y
                else:
                    X_test = np.concatenate((X_test, X))
                    y_test = np.concatenate((y_test, y))
                    
                frames = sleepy_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 1)
                
                X_test = np.concatenate((X_test, X))
                y_test = np.concatenate((y_test, y))

            X_train, y_train = self.shuffle_data(X_train, y_train)
            X_test, y_test = self.shuffle_data(X_test, y_test)

            return X_train, X_test, y_train, y_test
        
    def get_shallow_data_multi_sub(self, normal_frames_dict, leave_out_ratio=0.3):
            subjects = list(normal_frames_dict.keys())
            leave_out_subjects = np.random.choice(subjects, int(len(subjects) * leave_out_ratio), replace=False)
            print("Test subjects", leave_out_subjects)

            X_train, y_train = None, None
            for subject in subjects:
                if subject in leave_out_subjects:
                    continue
                frames = normal_frames_dict[subject]
                X = self.split_frames(frames)
                print(X.shape)
                assert 0
                y = self.create_labels(len(X), 0)
                if X_train is None:
                    X_train, y_train = X, y
                else:
                    X_train = np.concatenate((X_train, X))
                    y_train = np.concatenate((y_train, y))

                frames = sleepy_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 1)
                X_train = np.concatenate((X_train, X))
                y_train = np.concatenate((y_train, y))

            X_test, y_test = None, None
            for subject in leave_out_subjects:
                print('test', subject)
                frames = normal_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 0)

                if X_test is None:
                    X_test, y_test = X, y
                else:
                    X_test = np.concatenate((X_test, X))
                    y_test = np.concatenate((y_test, y))
                    
                frames = sleepy_frames_dict[subject]
                X = self.split_frames(frames)
                y = self.create_labels(len(X), 1)
                
                X_test = np.concatenate((X_test, X))
                y_test = np.concatenate((y_test, y))

            X_train, y_train = self.shuffle_data(X_train, y_train)
            X_test, y_test = self.shuffle_data(X_test, y_test)

            return X_train, X_test, y_train, y_test
    
    def train(self, X_train, y_train, num_epochs=10, batch_size=32, validation_split=0.2):
        self.batch_size = batch_size
        model = self.construct_model()
        early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

        model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=validation_split, callbacks=[early_stop])
        
        self.model = model
        print("Done with training")
        return model
    
    def test(self, X_test, y_test):
        model = self.model
        return model.evaluate(X_test, y_test, batch_size=self.batch_size)
    
    def get_f1(self, X_test, y_test):
        y_predict = model.predict(X_test, batch_size = self.batch_size)
        y_predict_int = np.round(y_predict).astype(int).flatten()
        f1 = f1_score(y_test_int, y_predict_int)
        return f1

In [18]:
detector = DrowsinessDetector(segment_length=60 * 46)
#normal_embs, sleepy_embs = load_embeddings('sub10')
#X_train, X_test, y_train, y_test = detector.get_data_one_sub(normal_embs, sleepy_embs)

subjects = [1,3,4,6,7,9,14,15,16,20,23,24]

leave_out_ratio = 0.3
leave_out_subjects = np.random.choice(subjects, int(len(subjects) * leave_out_ratio), replace=False)

normal_dict, sleepy_dict = load_multi_embeddings(subjects)
X_deep_train, X_deep_test, y_train, y_test = detector.get_deep_data(normal_dict, sleepy_dict, subjects, leave_out_subjects)

model = detector.train(X_train, y_train, num_epochs=100, batch_size=8)
#detector.test(X_test, y_test)
#y_test, y_predict = detector.compare_predictions(X_test, y_test)
#print(y_test, y_predict)

Test subjects ['15' '23' '16']
train 1
(62, 2760, 2048)


AssertionError: 

In [ ]:
import gc
gc.collect()
f1 = detector.get_f1(X_test, y_test)
print(f1)

## Shallow data retrieval

In [16]:
from helping_functions import *


def get_data(subject, treshhold, segment_length):
    status_rates_sleepy, wrong_frames_sleepy = load_blinks(subject, 'sleepy') 
    status_rates_normal, wrong_frames_normal = load_blinks(subject, 'normal') 
    
    print("Starting segmenting normal condition")
    blink_counts_normal, average_durs_normal = run_analysis(status_rates_normal, wrong_frames_normal, treshhold, segment_length)
    print("Starting segmenting sleepy condition")
    blink_counts_sleepy, average_durs_sleepy = run_analysis(status_rates_sleepy, wrong_frames_sleepy, treshhold, segment_length)
    
    return list(zip(blink_counts_normal, average_durs_normal)), list(zip(blink_counts_sleepy, average_durs_sleepy))

def split_into_segments(frames, gru_segment_length):
    num_segments = len(frames) // gru_segment_length
    frames = frames[:num_segments * gru_segment_length]
    return np.array(np.split(np.array(frames), num_segments))

def get_shallow_data(subjects, threshold, segment_length):
    for sub in subjects:
        subject = 'subject' + str(sub)
        X_normal, X_sleepy = get_data(subject, threshold, segment_length)
        X_normal_final = split_into_segments(X_normal, 6)
        print(X_normal_final.shape)
        print(X_normal_final)
        assert 0

subjects = [1,3,4,6,7,9,14,15,16,20,23,24]
treshhold = 10
segment_length = 46 * 10

get_shallow_data(subjects, treshhold, segment_length)


Starting segmenting normal condition
Number of segments 353
Starting segmenting sleepy condition
Number of segments 325
(58, 6, 2)
[[[ 6.          1.66666667]
  [ 6.          2.33333333]
  [ 7.          2.28571429]
  [ 5.          2.        ]
  [ 7.          1.57142857]
  [11.          2.09090909]]

 [[13.          1.84615385]
  [12.          2.41666667]
  [11.          1.90909091]
  [11.          1.90909091]
  [13.          2.15384615]
  [ 7.          2.14285714]]

 [[14.          2.14285714]
  [17.          2.76470588]
  [16.          2.        ]
  [27.          3.25925926]
  [34.          2.85294118]
  [27.          2.62962963]]

 [[30.          3.1       ]
  [17.          1.47058824]
  [27.          2.        ]
  [16.          1.25      ]
  [19.          2.15789474]
  [19.          3.21052632]]

 [[11.          2.72727273]
  [12.          2.        ]
  [12.          1.83333333]
  [11.          2.        ]
  [14.          2.28571429]
  [12.          2.        ]]

 [[ 9.          2.4

AssertionError: 